In [1]:
from dataset import *

import csv

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
model_ = models.resnet18(pretrained=False)
model_.fc = nn.Linear(512, 20)
model_.load_state_dict(torch.load('./best_single.pt'))

<All keys matched successfully>

In [3]:
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4633, 0.4887, 0.4060], std=[0.1824, 0.1601, 0.1958])])

val_dataset = NIPA_Dataset(data_type='val', transform=val_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
labels = []

model_.eval()

# Test
with torch.no_grad():
    for i, (X, y) in enumerate(val_loader):
        output = model_(X)
        label = torch.argmax(output, dim=1)
        labels += label.tolist()

In [5]:
label_dic = {0:'3_5', 1:'3_20', 2:'4_2', 3:'4_7', 4:'4_11', 5:'5_8', 6:'7_1', 7:'7_20', 8:'8_6', 9:'8_9', 10:'10_20',
             11:'11_14', 12:'13_1', 13:'13_6', 14:'13_9', 15:'13_15', 16:'13_16', 17:'13_17', 18:'13_18', 19:'13_20'}

with open('./result.tsv', 'a') as tsv_file:
    tsv_writer = csv.writer(tsv_file, delimiter='\t')
    for i, label in enumerate(labels):
        raw_label = label_dic[label]
        p_label, d_label = raw_label.split('_')
        tsv_writer.writerow([str(i)+'.jpg', p_label, d_label])